### Input data: 

1. Download all files present on this [link](https://figshare.com/articles/dataset/PPI_prediction_from_sequence_gold_standard_dataset/21591618/3) from Bernett et al., 2023

In [2]:
import os,sys,re
import argparse, json
import copy
import random
import pickle
import math
import torch
from torch import nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from tqdm import tqdm
#from tqdm.notebook import tqdm
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Polypeptide import one_to_index
from Bio.PDB import Selection
from Bio import SeqIO
from Bio.PDB.Residue import Residue
from easydict import EasyDict
import enum
import esm, gzip
from Bio import SeqIO
from esm.model.esm2 import ESM2
from collections import OrderedDict
from sklearn.metrics import mean_squared_error
import scipy.stats

In [2]:
fasta_dictionary = SeqIO.to_dict(SeqIO.parse('human_swissprot_oneliner.fasta', "fasta"))

def make_new_dfs(data_name):
    data_df_neg = pd.read_csv(f'{data_name}_neg_rr.txt', sep=' ', header=None)
    data_df_pos = pd.read_csv(f'{data_name}_pos_rr.txt', sep=' ', header=None)
    
    data_df = pd.concat([data_df_neg, data_df_pos], ignore_index=True)
    labels = [0]*len(data_df_neg) + [1]*len(data_df_pos)
    
    seq1 = []
    seq2 = []
    new_labels = []
    for index, row in data_df.iterrows():
        if row[0] in fasta_dictionary and row[1] in fasta_dictionary:
            seq1.append(str(fasta_dictionary[row[0]].seq))
            seq2.append(str(fasta_dictionary[row[1]].seq))
            new_labels.append(labels[index])
    seq_df = pd.DataFrame({'seq1': seq1, 'seq2': seq2, 'labels':new_labels})
    seq_df.to_csv(f'{data_name}_seqs.csv', index=False)
    print(f'{len(seq_df)} out of {len(data_df)} data points found')

make_new_dfs('Intra0')
make_new_dfs('Intra1')
make_new_dfs('Intra2')

59260 out of 59260 data points found
163019 out of 163192 data points found
52048 out of 52048 data points found
